# Copyright

<PRE>
Jelen iPython notebook a Budapesti Műszaki és Gazdaságtudományi Egyetemen tartott 
"Deep Learning a gyakorlatban Python és LUA alapon" tantárgy segédanyagaként készült a   
https://github.com/bradleypallen/keras-movielens-cf oldalon található forrás alapján.

A tantárgy honlapja: http://smartlab.tmit.bme.hu/oktatas-deep-learning 
Deep Learning kutatás: http://smartlab.tmit.bme.hu/deep-learning

A forráskódot MIT licensz védi.
</PRE>

## Importok

In [1]:
import os
import pandas as pd

## Tanítóadatok

In [2]:
# ! wget -q http://files.grouplens.org/datasets/movielens/ml-1m.zip#
# ! unzip ml-1m.zip

## Konstansok


In [3]:
BASE_DIR = '.' # Modify this if needed to the local directory that the MovieLens 1M Dataset has been unzipped into. 
MOVIELENS_DIR = BASE_DIR + '/ml-1m/'
USER_DATA_FILE = 'users.dat'
MOVIE_DATA_FILE = 'movies.dat'
RATING_DATA_FILE = 'ratings.dat'
AGES = { 1: "Under 18", 18: "18-24", 25: "25-34", 35: "35-44", 45: "45-49", 50: "50-55", 56: "56+" }
OCCUPATIONS = { 0: "other or not specified", 1: "academic/educator", 2: "artist", 3: "clerical/admin",
                4: "college/grad student", 5: "customer service", 6: "doctor/health care",
                7: "executive/managerial", 8: "farmer", 9: "homemaker", 10: "K-12 student", 11: "lawyer",
                12: "programmer", 13: "retired", 14: "sales/marketing", 15: "scientist", 16: "self-employed",
                17: "technician/engineer", 18: "tradesman/craftsman", 19: "unemployed", 20: "writer" }
RATINGS_CSV_FILE = 'ml1m_ratings.csv'
USERS_CSV_FILE = 'ml1m_users.csv'
MOVIES_CSV_FILE = 'ml1m_movies.csv'

##  MovieLens 1M adatok betöltése

A MovieLens 1M adathalmaz a http://files.grouplens.org/datasets/movielens/ml-1m.zip oldalon érhető el. Felhasználókról, filemkről tartalmaz adatokat (értékelést). Van olyan film, amelyhez nem tartozik értékelés.

In [5]:
ratings = pd.read_csv(os.path.join(MOVIELENS_DIR, RATING_DATA_FILE), 
                    sep='::', 
                    engine='python', 
                    encoding='latin-1',
                    names=['userid', 'movieid', 'rating', 'timestamp'])
max_userid = ratings['userid'].drop_duplicates().max()
max_movieid = ratings['movieid'].drop_duplicates().max()
ratings['user_emb_id'] = ratings['userid'] - 1
ratings['movie_emb_id'] = ratings['movieid'] - 1
print ('Értékelések száma: ', len(ratings))
ratings.to_csv(RATINGS_CSV_FILE, 
               sep='\t', 
               header=True, 
               encoding='latin-1', 
               columns=['userid', 'movieid', 'rating', 'timestamp', 'user_emb_id', 'movie_emb_id'])
print ('Értékelések mentve ide:', RATINGS_CSV_FILE)

Értékelések száma:  1000209
Értékelések mentve ide: ml1m_ratings.csv


In [6]:
users = pd.read_csv(os.path.join(MOVIELENS_DIR, USER_DATA_FILE), 
                    sep='::', 
                    engine='python', 
                    encoding='latin-1',
                    names=['userid', 'gender', 'age', 'occupation', 'zipcode'])
users['age_desc'] = users['age'].apply(lambda x: AGES[x])
users['occ_desc'] = users['occupation'].apply(lambda x: OCCUPATIONS[x])
print (len(users), 'betöltve az összes', max_userid, 'felhasználói profil közül.')
users.to_csv(USERS_CSV_FILE, 
             sep='\t', 
             header=True, 
             encoding='latin-1',
             columns=['userid', 'gender', 'age', 'occupation', 'zipcode', 'age_desc', 'occ_desc'])
print ('Felhasználói profilok mentve ide: ', USERS_CSV_FILE)

6040 betöltve az összes 6040 felhasználói profil közül.
Felhasználói profilok mentve ide:  ml1m_users.csv


In [7]:
movies = pd.read_csv(os.path.join(MOVIELENS_DIR, MOVIE_DATA_FILE), 
                    sep='::', 
                    engine='python', 
                    encoding='latin-1',
                    names=['movieid', 'title', 'genre'])
print (len(movies), 'betöltve az összes', max_movieid, 'filmprofil közül.')
movies.to_csv(MOVIES_CSV_FILE, 
              sep='\t', 
              header=True, 
              columns=['movieid', 'title', 'genre'])
print ('Filmprofilok mentve ide: ', MOVIES_CSV_FILE)

3883 betöltve az összes 3952 filmprofil közül.
Filmprofilok mentve ide:  ml1m_movies.csv


In [8]:
# Hány felhazsnálótól van egyáltalán adatunk
print (len(ratings['userid'].drop_duplicates()), 'felhasználó az összes', max_userid, 'közül értékelt legalább egy filmet.')

6040 felhasználó az összes 6040 közül étékelt legalább egy filmet.


In [12]:
# Mennyi értékelésünk van filmekre
print (len(ratings['movieid'].drop_duplicates()), 'film az összes', max_movieid, 'közül kapott értékelést.')

3706 film az összes 3952 közül kapott értékelést.
